In [ ]:
(setup-amber-paths)
(leap:source "leaprc.water.tip3p")
(leap:load-off "solvents.lib")
(leap:load-off "atomic_ions.lib")
(leap:load-atom-type-rules "ATOMTYPE_GFF.DEF")
(leap:source "leaprc.ff14SB.redq")
(leap:source "leaprc.gaff")

In [ ]:
*default-pathname-defaults*

In [ ]:
(setf *default-pathname-defaults* (merge-pathnames "data/amber-dynamics-2/" *default-pathname-defaults*))

In [ ]:
(defparameter tirun::*tiruns* (tirun:load-tiruns "postcharge/postcharge.cando"))

In [ ]:
(defparameter tirun::*receptor* (first (tirun:receptors tirun::*tiruns*)))

In [ ]:
(defparameter tirun::*side-name* :complex)

In [ ]:
(defparameter tirun::*morph* (tirun::find-morph-with-name "CHEMBL1089056-CHEMBL1088740" tirun::*tiruns*))

In [ ]:
(defparameter tirun::*source* (tirun:source tirun::*morph*))

In [ ]:
(defparameter tirun::*target* (tirun:target tirun::*morph*))

In [ ]:
(defparameter tirun::*system* (cando:combine (chem:matter-copy (tirun::molecule tirun::*target*))
    (chem:matter-copy tirun::*receptor*)))

In [ ]:
(leap:solvate-box tirun::*system*
                  (leap.core:lookup-variable 
                   (tirun::solvent-box tirun::*tiruns*))
                  (tirun::solvent-buffer tirun::*tiruns*)
                  :closeness
                  (tirun::solvent-closeness tirun::*tiruns*))
(leap.add-ions:add-ions tirun::*system* :|Cl-| 0)
(cando:save-mol2 tirun::*system* "complex.mol2")

In [ ]:
(leap.topology:save-amber-parm-format tirun::*system* "complex.parm7" "complex.rst7")

In [ ]:
(ql:quickload :amber)

In [ ]:
*default-pathname-defaults*

In [ ]:
(core:copy-file "complex.parm7" (ensure-directories-exist "jobs/complex.parm7"))
(core:copy-file "complex.rst7" "jobs/complex.rst7")

In [ ]:
(let* ((top #P"complex.parm7")
           (crd #P"complex.rst7")
           (start (amber:simple-jupyter-job :topology-file top :coordinate-file crd))
           (min (amber:minimize :previous-job start))
           (heat (amber:heat :previous-job min))
           (press (amber:pressurize :previous-job heat))
           (dynamics (amber:dynamics :previous-job press :nstlim 100000 :ntwx 1000)))
      (amber:generate-all-code start
                               (list (amber:mdcrd dynamics))
                               :pathname-defaults #P"jobs/")
   (defparameter *start* start))

In [ ]:
(with-open-file (sout "/tmp/simple.dot" :direction :output)
    (graphviz-amber:draw-graph-stream *start* sout))

# Now go into the jobs directory and run the makefile

In [ ]:
(setf *default-pathname-defaults* (merge-pathnames "data/amber-dynamics-2/jobs/" *default-pathname-defaults*))

In [ ]:
(defparameter *agg* (load-mol2 "jobs/complex-vdw-bonded.mol2"))

In [ ]:
(ql:quickload "netcdf")

In [ ]:
(ql:quickload :static-vectors)

In [ ]:
(ql:quickload :cando-jupyter)

In [ ]:
(probe-file "heat.nc")

In [ ]:
(:= *n* (netcdf::nc-open (probe-file "heat.nc") :mode netcdf-cffi:+nowrite+))

In [ ]:
(:= *nc* (cando-jupyter:make-amber-netcdf-trajectory :netcdf *n* :matter *agg*))

In [ ]:
(:= *v* (cando-jupyter:show-amber-netcdf-trajectory *nc*))

In [ ]:
*v*

In [ ]:
(nglv::center *v*)

In [ ]:
(nglv::clear-representations *v*)

In [ ]:
(nglv::add-representation *v* "ball+stick" :selection "protein")

In [ ]:
(nglv::add-representation *v* "ribbon" :selection "protein")